In [1]:
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_nature_pyscf
import qiskit_algorithms
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi
import pylatexenc
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import qiskit_addon_sqd

In [4]:
print(qiskit.__version__)
print(qiskit_aer.__version__)
print(qiskit_algorithms.__version__)
print(qiskit_nature.__version__)
print(qiskit_nature_pyscf.__version__)

1.4.0
0.17.0
0.3.1
0.7.2
0.4.0


In [5]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()

In [6]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(electronic_structure_problem.second_q_ops()[0])

In [7]:
from KQSCI_module import *

In [8]:
evolved_state_file = "state_probabilities.txt"
num_qubits = 12
state_probabilities = read_states(evolved_state_file)

In [10]:
# Iterate over each time step in state_probabilities
for time, counts in state_probabilities.items():
    # Perform SQD analysis for the current time step
    # sqd_result = sqd(counts)
    # print(f"Time: {time}, SQD Result: {sqd_result}")
    print(counts)

{'000011000011': 1.0}
{'000011000011': 0.99927, '100001100001': 0.00057, '100001000101': 5e-05, '000101100001': 5e-05, '001001001001': 1e-05, '010001010010': 1e-05, '010001010001': 1e-05, '000110000110': 2e-05, '001010001010': 1e-05}
{'000011000011': 0.99734, '100001100001': 0.00186, '000101100001': 0.00019, '010001010001': 0.00011, '100001000101': 0.00017, '000101000101': 1e-05, '001001001001': 9e-05, '100001000011': 4e-05, '000110000110': 5e-05, '100010100010': 2e-05, '000101000011': 2e-05, '000110000011': 4e-05, '010010010001': 2e-05, '001001001010': 1e-05, '001010001010': 2e-05, '000011000101': 1e-05}
{'000011000011': 0.99411, '100001100001': 0.00418, '000110000110': 0.00013, '100001000101': 0.00027, '000101100001': 0.00028, '010010010010': 1e-05, '010001010001': 0.00028, '000011100001': 3e-05, '010001010010': 2e-05, '001001001001': 0.00015, '000011000101': 5e-05, '000110000011': 6e-05, '100001000011': 0.00021, '000011000110': 8e-05, '000101000011': 3e-05, '100010000011': 3e-05, '0

In [18]:
from qiskit_addon_sqd.counts import counts_to_arrays
from qiskit_addon_sqd.qubit import solve_qubit

In [33]:
for time in list(state_probabilities.keys()):
    if time != 0:
        # Perform SQD analysis for the current time step
        counts = state_probabilities[time]
        # Perform SQD analysis for the current time step
        chis, coeff =  counts_to_arrays(counts)
        eignvalues, eignvector  = solve_qubit(chis, qubit_hamiltonian)
        # Print the results
        print(f"Time: {time}, Eigenvalues: {eignvalues[0]}")
    else:
        # Skip the first time step
        continue

Time: 0.17452777777777778, Eigenvalues: -8.879934969973807
Time: 0.34905555555555556, Eigenvalues: -8.880828915763843
Time: 0.5235833333333333, Eigenvalues: -8.880872040343506
Time: 0.6981111111111111, Eigenvalues: -8.880899043922081
Time: 0.872638888888889, Eigenvalues: -8.880896544655839
Time: 1.0471666666666666, Eigenvalues: -8.880904025037172
Time: 1.2216944444444444, Eigenvalues: -8.880909718931445
Time: 1.3962222222222223, Eigenvalues: -8.880907611166839
Time: 1.57075, Eigenvalues: -8.880909493201814


In [36]:
# Combine all dictionaries in state_probabilities into one dictionary
states_all = {}
for time, counts in state_probabilities.items():
    for key, value in counts.items():
        if key in states_all:
            states_all[key] += value
        else:
            states_all[key] = value


In [37]:

chis , coeff = counts_to_arrays(states_all)
eignvalues, eignvector  = solve_qubit(chis, qubit_hamiltonian)
eignvalues[0]

np.float64(-8.880918641282538)

In [38]:
eignvalues[0] + 0.998447567774

np.float64(-7.882471073508538)